# Coref resolution

In [2]:
import pickle

# load saved model
with open('allen_coref' , 'rb') as f:
    predictor = pickle.load(f)

text = "A cat is walking. It tripped and got hurt fell on a stone. It died. Dog just walked past. It died."

prediction = predictor.predict(document=text)  # get prediction
print("Clsuters:-")
for cluster in prediction['clusters']:
    print(cluster)  # list of clusters (the indices of spaCy tokens)
# Result: [[[0, 3], [26, 26]], [[34, 34], [50, 50]]]
print('\n\n') #Newline

coref_text = predictor.coref_resolved(text)

print('Coref resolved: ', coref_text)  # resolved text
# Result: Joseph Robinette Biden Jr. is an American politician who is the 46th and

c:\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Clsuters:-
[[0, 1], [5, 5], [15, 15]]
[[18, 18], [23, 23]]



Coref resolved:  A cat is walking. A cat tripped and got hurt fell on a stone. A cat died. Dog just walked past. Dog died.


# SAO parsing

In [3]:
# !pip install textacy

In [4]:
from subject_verb_object_extract import findSVOs, nlp
# coref_text = "A cat is walk rapidly on a sunny day. A cat fall on a stone. A cat dead"
tokens = nlp(coref_text)
SVOs = findSVOs(tokens)
print(coref_text)
print(SVOs)

C:\Users\shour\AppData\Roaming\Python\Python39\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


A cat is walking. A cat tripped and got hurt fell on a stone. A cat died. Dog just walked past. Dog died.
[('A cat', 'walking'), ('A cat', 'died'), ('Dog', 'walked'), ('Dog', 'died')]


In [5]:
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.tokenize import word_tokenize
  
stop_words = set(stopwords.words('english'))
svos = defaultdict(list)

for svo in SVOs:  
    word_tokens = word_tokenize(svo[0])
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    svos["".join(filtered_sentence)].append(svo[1])
    
svos = dict(svos)
print(svos)

charcaters = list(svos.keys())
print(charcaters)

{'cat': ['walking', 'died'], 'Dog': ['walked', 'died']}
['cat', 'Dog']


In [6]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

actions_movement = {'die' : 0, 'fall' : 0, 'hurt' : 0, 'idle' : 0, 'jump' : 1, 'run' : 1, 'slide' : 1, 'walk' : 1}
#word2vec similarity between the incoming action vs the ones we have in list of actions and then set a threshold and execute the action based on it

In [7]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\shour\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shour\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
import itertools
for actions in list(itertools.chain(*svos.values())):
    action = WordNetLemmatizer().lemmatize(actions,'v')
    print(action)

print(svos)


walk
die
walk
die
{'cat': ['walking', 'died'], 'Dog': ['walked', 'died']}


In [9]:
# !pip install pygame

In [12]:
import pygame
import glob
from time import sleep
 
SIZE = WIDTH, HEIGHT = 1280, 720 #the width and height of our screen
FPS = 9 #Frames per second

screen = pygame.display.set_mode(SIZE)
bg = pygame.transform.scale(pygame.image.load('pygame\sunny_day.png') , SIZE)
 
class MySprite(pygame.sprite.Sprite):
    def __init__(self, char, x, y):
        super(MySprite, self).__init__()
        
        self.images = []
        self.actions = []
        # self.images = [pygame.image.load(img) for img in glob.glob("pygame\\cat\\*.png")]
        for action in svos[char]:
            action = stemmer.stem(action)            
            self.actions.append(action)
            print(self.actions)
            
            self.images.extend([pygame.transform.scale(pygame.image.load(img) , (200,200)) for img in glob.glob("pygame\\" + char + "\\" + action + "*.png")])
            
        print(self.images)
        self.char = char
        self.index = 0
        self.action_count = -1
        self.dir = 'r'
        self.x = x
        self.y = y   

    def movement_update(self):
        if self.x == 880:
            self.dir = 'l'
        elif self.x == 0:
            self.dir = 'r'

        if self.dir == 'r':                        
            self.x += FPS
        else:
            self.image = pygame.transform.flip(self.image, True, False)
            self.x -= FPS
        screen.blit(self.image, (self.x % 880, self.y))
 
    def update(self):
        if self.index >= len(self.images):
            print("hello world")
            pygame.quit()
            return 0
        if self.index % 10 == 0:
            self.action_count += 1
        self.image = self.images[self.index]
        self.index += 1

        if actions_movement[self.actions[self.action_count]]:
            self.movement_update()
        else:
            screen.blit(self.image, (self.x, self.y))
        
        return 1
        
 
def main():
    pygame.init()
    pygame.display.set_caption("Trace")
    chars = []
    for i in range(len(charcaters)):
        chars.append(MySprite(charcaters[i], (200 * i), 350))
    # my_group = pygame.sprite.Group(my_sprite)
    clock = pygame.time.Clock()
    
    loop = 1
    while loop:   
        
        screen.fill((0,0,0))        
        screen.blit(bg, (0, 0))

        for i in chars:
            loop = i.update()        
        pygame.display.update()
        clock.tick(FPS)
    pygame.quit()
 
if __name__ == '__main__':
    main()

['walk']
['walk', 'die']
[<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>]
['walk']
['walk', 'die']
[<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200

error: video system not initialized

# NER

In [ ]:
import spacy 
import spacy.cli
# Download following once
#spacy.cli.download("en_core_web_sm")

In [ ]:
from spacy import displacy

nlp = spacy.load('en_core_web_sm')

In [ ]:
text = "Victoria was a major in CS. Victoria studies at MIT."
doc = nlp(text)

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print(ents)
# now we use displaycy function on doc2
displacy.render(doc, style='ent', jupyter=True)

[('CS', 24, 26, 'ORG'), ('MIT', 48, 51, 'ORG')]
